# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\pos_add_20240815.csv")
df.shape

(13, 8)

In [3]:
# df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data May042024.xlsx", sheet_name = 'Sheet1')
# df.head(10)

In [4]:
df.head()

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Prince San Remigio,Standalone,POS Operational,"August 15, 2024",MY841CR,Cebu North,Rona Jay Sinangote,Prince Retail
1,Prince Medellin,Standalone,POS Operational,"August 15, 2024",M237O5H,Cebu North,Rona Jay Sinangote,Prince Retail
2,Prince Daan Bantayan,Standalone,For Opening,"August 15, 2024",MXZEABL,Cebu North,Rona Jay Sinangote,Prince Retail
3,Prince Bantayan Island,Standalone,For Opening,"August 15, 2024",M9K1OHN,Cebu North,Rona Jay Sinangote,Prince Retail
4,Prince Asturias,Standalone,POS Operational,"August 15, 2024",MX8AOB7,Cebu South,Alexander Limosnero,Prince Retail


In [5]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [6]:
df['BrandStores'].value_counts()

BrandStores
Prince Retail    11
Rulls             1
Home Along        1
Name: count, dtype: int64

Check whether we have duplicate purple key in the data provided by the SIL team

In [7]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
MY841CR    1
M237O5H    1
MXZEABL    1
M9K1OHN    1
MX8AOB7    1
MT0X5GI    1
MGF2LMB    1
M0OEZGX    1
M4HY0T9    1
ME9BWUO    1
MNRVWJ1    1
M1I2WVE    1
MZKJOGD    1
Name: count, dtype: int64

In [8]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,Prince Retail,Prince San Remigio,Standalone,POS Operational,"August 15, 2024",MY841CR,Cebu North,Rona Jay Sinangote
1,Prince Retail,Prince Medellin,Standalone,POS Operational,"August 15, 2024",M237O5H,Cebu North,Rona Jay Sinangote
2,Prince Retail,Prince Daan Bantayan,Standalone,For Opening,"August 15, 2024",MXZEABL,Cebu North,Rona Jay Sinangote
3,Prince Retail,Prince Bantayan Island,Standalone,For Opening,"August 15, 2024",M9K1OHN,Cebu North,Rona Jay Sinangote
4,Prince Retail,Prince Asturias,Standalone,POS Operational,"August 15, 2024",MX8AOB7,Cebu South,Alexander Limosnero
5,Prince Retail,Prince Minglanilla,Standalone,POS Operational,"August 15, 2024",MT0X5GI,Cebu South,Alexander Limosnero
6,Prince Retail,Prince Badian,Standalone,POS Operational,"August 15, 2024",MGF2LMB,Cebu South,Alexander Limosnero
7,Prince Retail,Prince Sibonga,Standalone,For Opening,"August 15, 2024",M0OEZGX,Cebu South,Alexander Limosnero
8,Prince Retail,Prince Argao,Standalone,POS Operational,"August 15, 2024",M4HY0T9,Cebu South,Alexander Limosnero
9,Prince Retail,Prince Dalaguete,Standalone,For Opening,"August 15, 2024",ME9BWUO,Cebu South,Alexander Limosnero


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      13 non-null     object
 1   POS NAME         13 non-null     object
 2   STORE LOCATION   13 non-null     object
 3   STATUS           13 non-null     object
 4   DATE ACTIVATION  13 non-null     object
 5   PURPLE KEY       13 non-null     object
 6   REGION           13 non-null     object
 7   TSM              13 non-null     object
dtypes: object(8)
memory usage: 960.0+ bytes


In [10]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [11]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Prince Retail,Prince San Remigio,Standalone,POS Operational,"August 15, 2024",MY841CR,Cebu North,Rona Jay Sinangote
1,Prince Retail,Prince Medellin,Standalone,POS Operational,"August 15, 2024",M237O5H,Cebu North,Rona Jay Sinangote
2,Prince Retail,Prince Daan Bantayan,Standalone,For Opening,"August 15, 2024",MXZEABL,Cebu North,Rona Jay Sinangote
3,Prince Retail,Prince Bantayan Island,Standalone,For Opening,"August 15, 2024",M9K1OHN,Cebu North,Rona Jay Sinangote
4,Prince Retail,Prince Asturias,Standalone,POS Operational,"August 15, 2024",MX8AOB7,Cebu South,Alexander Limosnero


In [12]:
from datetime import datetime
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%m-%Y')
# Format the datetime object to YYYY-MM-DD
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')

ValueError: time data "August 15, 2024" doesn't match format "%d-%m-%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [13]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%B %d, %Y')


In [14]:
print(df['DateOpened'].dtype)
# datetime64[ns]
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

datetime64[ns]
2024-08-15 00:00:00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    13 non-null     object
 1   POS            13 non-null     object
 2   StoreLocation  13 non-null     object
 3   Status         13 non-null     object
 4   DateOpened     13 non-null     object
 5   PurpleKey      13 non-null     object
 6   Region         13 non-null     object
 7   TSM            13 non-null     object
dtypes: object(8)
memory usage: 960.0+ bytes


In [15]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

ValueError: time data "2024-08-15" doesn't match format "%d-%b-%y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [16]:
print(df['DateOpened'].dtype)
# datetime64[ns]

object


In [17]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-08-15


In [18]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

AttributeError: Can only use .dt accessor with datetimelike values

In [19]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    13 non-null     object        
 1   POS            13 non-null     object        
 2   StoreLocation  13 non-null     object        
 3   Status         13 non-null     object        
 4   DateOpened     13 non-null     datetime64[ns]
 5   PurpleKey      13 non-null     object        
 6   Region         13 non-null     object        
 7   TSM            13 non-null     object        
dtypes: datetime64[ns](1), object(7)
memory usage: 960.0+ bytes


In [21]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Prince Retail,Prince San Remigio,Standalone,POS Operational,2024-08-15,MY841CR,Cebu North,Rona Jay Sinangote
1,Prince Retail,Prince Medellin,Standalone,POS Operational,2024-08-15,M237O5H,Cebu North,Rona Jay Sinangote
2,Prince Retail,Prince Daan Bantayan,Standalone,For Opening,2024-08-15,MXZEABL,Cebu North,Rona Jay Sinangote
3,Prince Retail,Prince Bantayan Island,Standalone,For Opening,2024-08-15,M9K1OHN,Cebu North,Rona Jay Sinangote
4,Prince Retail,Prince Asturias,Standalone,POS Operational,2024-08-15,MX8AOB7,Cebu South,Alexander Limosnero


In [22]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [23]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=e7bb16f6-3039-4a24-9090-e658326cbbc6>

In [24]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID a30c644d-889c-4d83-a042-aeff724aff1e successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of d is:	(872, 8)


In [25]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID aeabf272-b30e-4dd7-b903-afc18060a876 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [26]:
dfsil.BrandStores.value_counts().sort_index()

BrandStores
A2RJ                            3
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       109
Faith Glorious                  9
Faith Glorious Fit              3
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     95
Home Factory                    2
Homeworks                      15
JSL                             7
JTG                             1
Jhunpyo                         8
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     54
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Prince and Aviel                3
RC

In [27]:
dfsil.shape

(872, 8)

In [28]:
dfsil.columns

Index(['BrandStores', 'POS', 'StoreLocation', 'Status', 'DateOpened',
       'PurpleKey', 'Region', 'TSM'],
      dtype='object')

In [29]:
dfsil[dfsil['PurpleKey'] =='M1I2WVE']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM


In [30]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 885 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [31]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID 41d3ca57-d075-4c84-98aa-2e9dcce93cd5 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (885, 8)


In [32]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
875,Prince Retail,Prince San Remigio,Standalone,POS Operational,2024-08-15,MY841CR,Cebu North,Rona Jay Sinangote
876,Prince Retail,Prince Medellin,Standalone,POS Operational,2024-08-15,M237O5H,Cebu North,Rona Jay Sinangote
877,Prince Retail,Prince Sibonga,Standalone,For Opening,2024-08-15,M0OEZGX,Cebu South,Alexander Limosnero
878,Prince Retail,Prince Dalaguete,Standalone,For Opening,2024-08-15,ME9BWUO,Cebu South,Alexander Limosnero
879,Prince Retail,Prince Asturias,Standalone,POS Operational,2024-08-15,MX8AOB7,Cebu South,Alexander Limosnero
880,Prince Retail,Prince Minglanilla,Standalone,POS Operational,2024-08-15,MT0X5GI,Cebu South,Alexander Limosnero
881,Prince Retail,Prince Badian,Standalone,POS Operational,2024-08-15,MGF2LMB,Cebu South,Alexander Limosnero
882,Prince Retail,Prince Argao,Standalone,POS Operational,2024-08-15,M4HY0T9,Cebu South,Alexander Limosnero
883,Rulls,Audionet Inline Gaisano Oslob,Gaisano Mall Oslob,POS Operational,2024-08-15,M1I2WVE,Cebu South,Alexander Limosnero
884,Home Along,Home Along Talavera,Standalone,POS Operational,2024-08-15,MZKJOGD,North Luzon,Rhea Geronimo


In [33]:
df['BrandStores'].value_counts().sort_index()

BrandStores
A2RJ                            3
Affordables                     2
Ansons                         15
ArkHome                         4
Automatic Centre               11
Chrisrence Home Appliances      2
Dizon Furniture                 1
ESLA                           51
FC Home                       109
Faith Glorious                  9
Faith Glorious Fit              3
Gadgetmate                     10
Hardware Sugar                  2
Hazrat                          6
Home Along                     96
Home Factory                    2
Homeworks                      15
JSL                             7
JTG                             1
Jhunpyo                         8
Junliz                          8
KUM Home Center                 3
Kservico                       10
MemoXpress                     54
PHPS                            2
Power Mac                       5
Power Mac Center                5
Presnet                        21
Prince Retail                  11
Pr

In [34]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,4,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,5,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
29,836,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
28,834,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
30,841,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
31,842,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
19,752,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
20,753,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
25,771,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo
22,768,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo


In [35]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [37]:
df[df['PurpleKey']== 'M1I2WVE']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
883,Rulls,Audionet Inline Gaisano Oslob,Gaisano Mall Oslob,POS Operational,2024-08-15,M1I2WVE,Cebu South,Alexander Limosnero


In [38]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=2a1b9a66-30a5-4948-bf19-159b5990e598>

In [39]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=ecd8b6cb-4f24-4b28-b803-f8457ea0274b>

In [40]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=1f8bb854-e4ba-4def-836a-de1ea415cbd8>

In [41]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID 014f6f57-a838-4b63-9fd9-bc04b02c5584 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(885, 8)

In [42]:
df[df['BrandStores']=='VMax Mobile'].sort_values(by='DateOpened')

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
512,VMax Mobile,Delle General Merch Harmony Mall,Harmony Village Mall,POS Operational,2024-07-27,M3YOW2R,South Luzon,Aubrey Lalong Isip
513,VMax Mobile,Vmax Central Binan 2,Central Mall Binan,POS Operational,2024-07-27,ME4NBT7,South Luzon,Aubrey Lalong Isip
514,VMax Mobile,Vmax Central Balibago,Victory Central Mall Balibago,POS Operational,2024-07-27,MWN8FVI,South Luzon,Aubrey Lalong Isip
515,VMax Mobile,Vmax Central Binan,Central Mall Binan,POS Operational,2024-07-27,MZEO85E,South Luzon,Aubrey Lalong Isip
516,VMax Mobile,Lets Mobile Phone and accessories Store,Divimall Cabuyao,POS Operational,2024-07-27,MSNZ3XL,South Luzon,Aubrey Lalong Isip
517,VMax Mobile,Vmax Waltermart Sta Rosa,Waltermart Starosa,POS Operational,2024-07-27,MJ0WZAK,South Luzon,Aubrey Lalong Isip
518,VMax Mobile,Realme Target Mall Balibago,Target Mall Balibago,For Opening,2024-08-02,MXACV0S,South Luzon,Aubrey Lalong Isip
519,VMax Mobile,Delle General Merch Target Mall,Target Mall Balibago,For Opening,2024-08-02,MLX9FJP,South Luzon,Aubrey Lalong Isip
520,VMax Mobile,St Vincent Telecom Target Mall,Target Mall Starosa,For Opening,2024-08-02,MVTZGDH,South Luzon,Aubrey Lalong Isip
521,VMax Mobile,Vmax Waltermart Cabuyao,Waltermart Cabuyao,For Opening,2024-08-02,MKPNC4B,South Luzon,Aubrey Lalong Isip


In [43]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center Pasig,Standalone,POS Operational,2024-08-02,MOJ34ME,ENCR,Martin Lin
2,Faith Glorious Fit,Event POS Faith Glorious Fit Sales Center San ...,Standalone,POS Operational,2024-08-02,MB3YSES,CAMANAVA,Paul Palonpon
3,Prince Retail,Prince Lopez,Standalone,For Opening,2024-08-15,MNRVWJ1,Bicol,Penfrancia Ayo
4,Home Along,Home Along Talavera,Standalone,POS Operational,2024-08-15,MZKJOGD,North Luzon,Rhea Geronimo
5,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
6,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
7,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
8,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
9,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None


In [44]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
786,Power Mac,Power Mac Center Greenbelt,Ayala Mall Circuit,For Opening,2024-08-16,M9T2W5I,NCR South 1,Klein Justine Tamayo
171,Power Mac,Power Mac Center Manila Bay,Ayala Mall Manila Bay,For Opening,2024-08-16,MBJZIEO,NCR South 2,Grace Belarmino
170,Power Mac,Power Mac Center Festival Supermall,Festival Supermall,For Opening,2024-08-16,MUVWA6D,NCR South 2,Grace Belarmino
270,Power Mac,Power Mac Center Robinsons Magnolia,Robinsons Magnolia,For Opening,2024-08-16,MW8XV42,NCR North,Sander Maghirang
677,Prince Retail,Prince Argao,Standalone,POS Operational,2024-08-15,M4HY0T9,Cebu South,Alexander Limosnero
591,Prince Retail,Prince Bantayan Island,Standalone,For Opening,2024-08-15,M9K1OHN,Cebu North,Rona Jay Sinangote
590,Prince Retail,Prince San Remigio,Standalone,POS Operational,2024-08-15,MY841CR,Cebu North,Rona Jay Sinangote
592,Prince Retail,Prince Medellin,Standalone,POS Operational,2024-08-15,M237O5H,Cebu North,Rona Jay Sinangote
676,Rulls,Audionet Inline Gaisano Oslob,Gaisano Mall Oslob,POS Operational,2024-08-15,M1I2WVE,Cebu South,Alexander Limosnero
678,Prince Retail,Prince Sibonga,Standalone,For Opening,2024-08-15,M0OEZGX,Cebu South,Alexander Limosnero


In [45]:
df[df['PurpleKey'] =='M1I2WVE']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
676,Rulls,Audionet Inline Gaisano Oslob,Gaisano Mall Oslob,POS Operational,2024-08-15,M1I2WVE,Cebu South,Alexander Limosnero


In [46]:
df.groupby(['BrandStores', 'DateOpened'])['PurpleKey'].nunique()

BrandStores                 DateOpened
A2RJ                        2024-02-05      3
Affordables                 2024-01-22      1
                            2024-04-12      1
Ansons                      2024-04-15     15
ArkHome                     2024-06-07      4
Automatic Centre            2023-07-15      6
                            2023-07-16      1
                            2023-07-22      4
Chrisrence Home Appliances  2024-07-13      2
Dizon Furniture             2022-07-23      1
ESLA                        2024-02-15     30
                            2024-04-22     21
FC Home                     2023-08-01     48
                            2023-09-01     20
                            2023-09-05      1
                            2023-09-14     25
                            2023-09-15     11
                            2023-11-10      1
                            2024-01-15      1
                            2024-03-26      1
                            2024-04-24   

In [47]:
df[df['BrandStores']=='Rulls']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
11,Rulls,Techbox Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8FWM1N,NCR East,Albee David
12,Rulls,Realme Ayala Marikina,Ayala Malls Marikina,For Opening,2024-04-15,MS1XI4A,NCR East,Albee David
13,Rulls,Rulls Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8U05KZ,NCR East,Albee David
14,Rulls,Mobilecraze K1 PG Marikina,Puregold Marikina,For Opening,2024-04-15,M9KLTD1,NCR East,Albee David
15,Rulls,Mobilecraze K3 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MW6HOE7,NCR East,Albee David
17,Rulls,Mobile Craze PG Tanay Rizal,Puregold Tanay Rizal,For Opening,2024-04-15,MFCWRJQ,NCR East,Albee David
19,Rulls,Fun Mobile Ayala Marikina,Ayala Malls Marikina,For Opening,2024-04-15,MK4CZP2,NCR East,Albee David
20,Rulls,Mobilecraze K2 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MUFDX4A,NCR East,Albee David
21,Rulls,MBC Montalban Town Center,Montalban Town Center,For Opening,2024-04-15,MI9KLNA,NCR East,Albee David
23,Rulls,Oppo Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MTEHX0R,NCR East,Albee David


In [48]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\POS_mapping_data.xlsx", index = False)

In [49]:
df[df['PurpleKey']=='M1I2WVE']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
676,Rulls,Audionet Inline Gaisano Oslob,Gaisano Mall Oslob,POS Operational,2024-08-15,M1I2WVE,Cebu South,Alexander Limosnero
